# Entrega 1
- Descripción del sistema computacional:
- Pregunta de estudio: 
- Tipos de datos a recolectar:

# Entrega 2
- Metodología de evaluación de desempeño:
    (Monitoreo, benchmarking o redes de colas)
- Diseño del estudio (y primeros resultados):

# Entrega 3
Informe final con
- Recoleccion de Datos
- Resultados
> con interpretacion
- Conclusiones

$$\text{Lista Nueva lista de Data}$$

<img src="../Shared/newData/list.png" alt="lista_newData">

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [12]:
def parseReqMemString(s):
    suf = ""
    i = len(s)-1
    while i and not s[i].isnumeric():
        suf += s[i]
        i-=1
    num = int(s[:i+1])
    if "G" in suf:
        num *= 1000
    elif "M" not in suf:
        print(suf) # Error
    return num

def parseData(x):
    num = 0
    c = 1000
    x_inv = x[::-1]
    while not x_inv[0].isnumeric():
        c = 1
        x_inv = x_inv[1:]

    num = float(x_inv[::-1])/c
    return float(x_inv[::-1])

def readData(filename, sep=",", parse_dates=[]):
    data = pd.read_csv(filename, sep=sep)
    for col in parse_dates:
        data[col] = pd.to_datetime(data[col], errors="coerce")
    data["ReqMem"] = data["ReqMem"].apply(
        lambda x: 0 if (len(x) == 2) else parseReqMemString(x)
    )
    return data  
    
def hCumsum(data):
    columns = data.columns[1:]
    data[columns] = data[columns].cumsum(axis=1)
    return data, columns

In [13]:
# 1. Read and clean data

# newRoute = "../Shared/oldData"
# # 1.1 WebData
# archivos = [
#     newRoute + "/" + "CPU Utilization.csv", #0
#     newRoute + "/" + "GPU Utilization.csv", #1
#     newRoute + "/" + "GPU Used Memory.csv", #2
#     newRoute + "/" + "Memory.csv",          #3
#     newRoute + "/" + "Patagon - Power.csv", #4
# ]

# data = []
# cols_ix = [] 
# for i, a in enumerate(archivos):
#     d, c = hCumsum(readData(a, parse_dates=["Time"]))
#     data.append(d)
#     cols_ix.append(c)
    
# 1.2 PatagonData
monitoreo = readData("../Shared/monitoreo.csv", sep="|", parse_dates=['Start', 'End'])

In [14]:
filtrado = [
    'Unnamed: 19',
    'ReqTRES'
]

In [15]:
monitoreo = monitoreo.copy()
monitoreo.drop([0, 1, 2], axis=0, inplace=True)
monitoreo.drop(filtrado, axis=1, inplace=True)

In [16]:
monitoreo.dropna(inplace=True)

In [17]:
monitoreo

,State,Start,End,Elapsed,ReqMem,AveDiskRead,MaxDiskRead,AveDiskWrite,MaxDiskWrite,AveRSS,MaxRSS,AveVMSize,MaxVMSize,ReqCPUS,CPUTime,CPUTimeRAW,MinCPU,TotalCPU
4,FAILED,2021-05-01 16:30:15,2021-05-01 16:30:16,00:00:01,100000,0,0,0,0,1260K,1260K,207696K,207696K,32,00:00:32,32,00:00:00,00:00.014
5,FAILED,2021-05-01 16:30:15,2021-05-01 16:30:16,00:00:01,100000,0,0,0,0,816K,816K,273752K,273752K,32,00:00:32,32,00:00:00,00:00.002
7,FAILED,2021-05-01 16:34:16,2021-05-01 16:34:17,00:00:01,100000,0,0,0,0,1264K,1264K,207696K,207696K,32,00:00:32,32,00:00:00,00:00.013
8,FAILED,2021-05-01 16:34:17,2021-05-01 16:34:17,00:00:00,100000,0,0,0,0,816K,816K,273752K,273752K,32,00:00:00,0,00:00:00,00:00.002
9,COMPLETED,2021-05-01 20:50:29,2021-05-01 20:50:30,00:00:01,100000,0,0,0,0,832K,832K,273752K,273752K,32,00:00:32,32,00:00:00,00:00.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15403,COMPLETED,2021-07-05 03:12:28,2021-07-05 03:12:31,00:00:03,65556,0,0,0,0,860K,860K,273752K,273752K,1,00:02:24,144,00:00:00,00:01.265
15407,FAILED,2021-07-05 09:33:11,2021-07-05 09:33:12,00:00:01,0,0,0,0,0,864K,864K,273752K,273752K,8,00:00:08,8,00:00:00,00:00.004
15408,COMPLETED,2021-07-05 09:33:12,2021-07-05 09:33:14,00:00:02,0,0,0,0,0,856K,856K,273752K,273752K,8,00:00:16,16,00:00:00,00:00.821
15409,COMPLETED,2021-07-05 09:33:14,2021-07-05 09:33:15,00:00:01,0,0,0,0,0,860K,860K,273752K,273752K,8,00:00:08,8,00:00:00,00:00.776


In [18]:
data_toparse = ["AveDiskRead",
                "MaxDiskRead",
                "AveDiskWrite",
                "MaxDiskWrite",
                "AveRSS",
                "MaxRSS",
                "AveVMSize",
                "MaxVMSize"
               ]

for col in data_toparse:
    monitoreo[col] = monitoreo[col].apply(
        lambda x: 0 if (len(x) == 2) else parseData(x)
    )

In [19]:
monitoreo

,State,Start,End,Elapsed,ReqMem,AveDiskRead,MaxDiskRead,AveDiskWrite,MaxDiskWrite,AveRSS,MaxRSS,AveVMSize,MaxVMSize,ReqCPUS,CPUTime,CPUTimeRAW,MinCPU,TotalCPU
4,FAILED,2021-05-01 16:30:15,2021-05-01 16:30:16,00:00:01,100000,0.0,0.0,0.0,0.0,1260.0,1260.0,207696.0,207696.0,32,00:00:32,32,00:00:00,00:00.014
5,FAILED,2021-05-01 16:30:15,2021-05-01 16:30:16,00:00:01,100000,0.0,0.0,0.0,0.0,816.0,816.0,273752.0,273752.0,32,00:00:32,32,00:00:00,00:00.002
7,FAILED,2021-05-01 16:34:16,2021-05-01 16:34:17,00:00:01,100000,0.0,0.0,0.0,0.0,1264.0,1264.0,207696.0,207696.0,32,00:00:32,32,00:00:00,00:00.013
8,FAILED,2021-05-01 16:34:17,2021-05-01 16:34:17,00:00:00,100000,0.0,0.0,0.0,0.0,816.0,816.0,273752.0,273752.0,32,00:00:00,0,00:00:00,00:00.002
9,COMPLETED,2021-05-01 20:50:29,2021-05-01 20:50:30,00:00:01,100000,0.0,0.0,0.0,0.0,832.0,832.0,273752.0,273752.0,32,00:00:32,32,00:00:00,00:00.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15403,COMPLETED,2021-07-05 03:12:28,2021-07-05 03:12:31,00:00:03,65556,0.0,0.0,0.0,0.0,860.0,860.0,273752.0,273752.0,1,00:02:24,144,00:00:00,00:01.265
15407,FAILED,2021-07-05 09:33:11,2021-07-05 09:33:12,00:00:01,0,0.0,0.0,0.0,0.0,864.0,864.0,273752.0,273752.0,8,00:00:08,8,00:00:00,00:00.004
15408,COMPLETED,2021-07-05 09:33:12,2021-07-05 09:33:14,00:00:02,0,0.0,0.0,0.0,0.0,856.0,856.0,273752.0,273752.0,8,00:00:16,16,00:00:00,00:00.821
15409,COMPLETED,2021-07-05 09:33:14,2021-07-05 09:33:15,00:00:01,0,0.0,0.0,0.0,0.0,860.0,860.0,273752.0,273752.0,8,00:00:08,8,00:00:00,00:00.776


In [20]:
monitoreo.describe()

,ReqMem,AveDiskRead,MaxDiskRead,AveDiskWrite,MaxDiskWrite,AveRSS,MaxRSS,AveVMSize,MaxVMSize,ReqCPUS,CPUTimeRAW
count,14488.000000,1.448800e+04,1.448800e+04,1.448800e+04,1.448800e+04,1.448800e+04,1.448800e+04,1.448800e+04,1.448800e+04,14488.000000,1.448800e+04
mean,22391.247929,2.410134e+03,2.410134e+03,3.612429e+02,3.612433e+02,3.276008e+06,3.275509e+06,2.159649e+07,8.899824e+06,7.635629,8.658586e+03
std,64672.597064,6.819272e+04,6.819272e+04,1.279443e+04,1.279443e+04,1.029663e+07,1.029665e+07,1.111510e+09,2.489614e+07,8.811634,1.353547e+05
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.308500e+03,2.308500e+03,1.000000,0.000000e+00
25%,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.560000e+02,8.560000e+02,2.737520e+05,2.737520e+05,1.000000,1.600000e+01
50%,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.600000e+02,8.600000e+02,2.737520e+05,2.737520e+05,8.000000,8.800000e+01
75%,65556.000000,7.000000e-02,7.000000e-02,0.000000e+00,0.000000e+00,3.664960e+05,3.573850e+05,4.700754e+06,4.700754e+06,8.000000,2.640000e+02
max,958444.000000,5.215385e+06,5.215385e+06,1.237126e+06,1.237126e+06,1.097190e+08,1.097190e+08,1.136220e+11,2.812504e+08,64.000000,8.575200e+06


In [ ]:
# completos = filtered_monitoreo[filtered_monitoreo["State"] == "COMPLETED"]
# fallidos = filtered_monitoreo[filtered_monitoreo["State"] == "FAILED"]
# corriendos = filtered_monitoreo[filtered_monitoreo["State"] == "RUNNING"]
# cancelados = filtered_monitoreo[filtered_monitoreo["State"] == "CANCELLED"]
# # dataframe[dataframe['Percentage'] > 70]

In [ ]:
# plt.figure(figsize=(15, 6), tight_layout=True)
# plt.bar([1,2,3], [1,2,3])

In [ ]:
# for i in range(len(data)):
#     plt.figure(figsize=(15, 10))
#     plt.plot(data[i]["Time"], data[i][cols_ix[i]])
#     plt.savefig("{}.png".format(archivos[i]), bbox_inches='tight')